# Cours de Python - Utilisation de l'API IIIF et manipulation de données 2

---

## Table des Matières

1. [Révisions](#révisions)
    - [Structure des manifestes](#structure-des-manifestes)
        - [Les principales composantes](#les-principales-composantes)
        - [Les canvas](#les-canvas)
        - [Trouver l'URL de l'image](#trouver-lurl-de-limage)
    - [Bibliothèque JSON](#bibliothèque-json)
        - [Rappel sur la bibliothèque](#rappel-sur-la-bibliothèque)
        - [Ouvrir un fichier JSON](#ouvrir-un-fichier-json)
        - [Écrire et enregistrer un fichier JSON](#écrire-et-enregistrer-un-fichier-json)
    - [La bibliothèque `os`](#la-bibliothèque-os)
        - [Intérêt de `os`](#intérêt-de-os)
        - [Création de filepath](#création-de-filepath)
        - [Création de dossier](#création-de-dossier)
2. [Télécharger des données avec la bibliothèque `requests`](#télécharger-des-données-avec-la-bibliothèque-requests)
    - [Intérêt de la bibliothèque](#intérêt-de-la-bibliothèque)
    - [Manipulation des codes et gestion des exceptions](#manipulation-des-codes-et-gestion-des-exceptions)
    - [Méthode `get` et gérer les `status_code`](#méthode-get-et-gérer-les-status_code)
    - [Téléchargement avec `requests'](#téléchargement-avec-requests)
3. [Manipuler des images](#manipuler-des-images)
    - [La bibliothèque PIL](#la-bibliothèque-pil)
    - [Manipulation de base d'une image](#manipulation-de-base-dune-image)
    - [Récupérer les données des images](#récupérer-les-données-des-images)

---

## Révisions

### Structure des manifestes

#### Les principales composantes

Un **manifeste** est un document structuré qui décrit une ressource numérique complexe. Il est souvent utilisé pour représenter des collections d'images, de textes ou de médias. Dans le contexte de l'histoire numérique, les manifestes facilitent l'accès et l'interopérabilité des ressources historiques en ligne.

- **Identifiant unique (`@id`)** : Un URL unique qui identifie le manifeste.
- **Type (`@type`)** : Indique le type de ressource (par exemple, "sc:Manifest").
- **Métadonnées (`metadata`)** : Informations sur la ressource, telles que le titre, le créateur, la date, etc.
- **Séquences (`sequences`)** : Liste des séquences de contenu, généralement une seule séquence nommée "normal".
- **Canvases (`canvases`)** : Représente les unités de contenu individuelles, comme les pages d'un livre, les folios d'un manuscrit ou les images d'une collection.

Exemple d'un manifeste simplifié au format JSON :


```json
{
  "@context": "http://iiif.io/api/presentation/2/context.json",
  "@id": "http://example.org/manifest.json",
  "@type": "sc:Manifest",
  "label": "Titre de la ressource",
  "metadata": [
    {
      "label": "Créateur",
      "value": "Nom du créateur"
    }
  ],
  "sequences": [
    {
      "@type": "sc:Sequence",
      "canvases": [
        {
          "@id": "http://example.org/canvas/p1",
          "@type": "sc:Canvas",
          "label": "Page 1",
          "images": [
            {
              "@type": "oa:Annotation",
              "motivation": "sc:painting",
              "resource": {
                "@id": "http://example.org/image/p1/full/full/0/default.jpg",
                "@type": "dctypes:Image",
                "format": "image/jpeg",
                "width": 1000,
                "height": 1500
              },
              "on": "http://example.org/canvas/p1"
            }
          ]
        }
      ]
    }
  ]
}
```

#### Les canvas

Un **canvas** représente une unité de contenu, comme une page ou une image individuelle. Il est essentiel pour structurer et naviguer à travers une collection ou un document complexe.

Propriétés d'un canvas :

- **Identifiant (`@id`)** : URL unique du canvas.
- **Type (`@type`)** : Généralement "sc:Canvas".
- **Label** : Nom ou titre du canvas.
- **Dimensions** : Largeur et hauteur du canvas.
- **Images** : Liste des annotations d'images associées au canvas. L'annotation dans le contexte de IIIF contient des informations sur l'image, telles que :
    - @type : Type de l'annotation, par exemple, "Annotation".
    - motivation : But de l'annotation. Pour une image, c'est souvent "sc:painting", ce qui signifie qu'elle est peinte ou placée sur le canvas.
    - resource : La ressource multimédia, souvent un lien vers l'image elle-même. Il peut contenir des détails sur la taille de l'image et des informations de type MIME (comme image/jpeg).
    - on : Référence au canvas sur lequel l'image doit être affichée. Cela associe l'image à un endroit spécifique sur ce canvas.

```json
{
    "@id": "https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/canvas/bbb-0318_001r.json",
    "@type": "sc:Canvas",
    "label": "1r",
    "height": 6496,
    "width": 4872,
    "images": [
        {
            "@id": "https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/annotation/bbb-0318_001r.json",
            "@type": "oa:Annotation",
            "motivation": "sc:painting",
            "on": "https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/canvas/bbb-0318_001r.json",
            "resource": {
                "@id": "https://www.e-codices.unifr.ch/loris/bbb/bbb-0318/bbb-0318_001r.jp2/full/full/0/default/jpg",
                "@type": "dctypes:Image",
                "format": "image/jpeg",
                "height": 6496,
                "width": 4872,
                "service": {
                    "@context": "http://iiif.io/api/image/2/context.json",
                    "@id": "https://www.e-codices.unifr.ch/loris/bbb/bbb-0318/bbb-0318_001r.jp2",
                    "profile": "http://iiif.io/api/image/2/level2.json"
                }
            }
        }
    ]
}

#### Trouver l'URL de l'image

Pour extraire l'URL d'une image spécifique à partir d'un manifeste, vous devez naviguer à travers la structure imbriquée du JSON.

Exemple en Python :

```python
import json

# Charger le manifeste
with open('manifest.json', 'r') as file:
    manifest = json.load(file)

# Accéder au premier canvas
canvas = manifest['sequences'][0]['canvases'][0]

# Accéder à l'annotation de l'image
annotation = canvas['images'][0]

# URL de l'image
image_url = annotation['resource']['@id']

print("URL de l'image :", image_url)
```

In [33]:
import json

def find_url_img(iiif_manifest):
    with open(iiif_manifest, 'r') as file:
        manifest = json.load(file)

    # Accéder au premier canvas
    canvas = manifest['sequences'][0]['canvases'][0]

    # Accéder à l'annotation de l'image
    annotation = canvas['images'][0]

    # URL de l'image
    image_url = annotation['resource']['@id']

    print("URL de l'image :", image_url)

---

### Bibliothèque JSON

#### Rappel sur la bibliothèque

La bibliothèque `json` est intégrée à Python et permet de travailler facilement avec des données au format JSON.

- **JSON** (JavaScript Object Notation) est un format léger d'échange de données, facile à lire et à écrire pour les humains, et facile à analyser et générer pour les machines.

#### Ouvrir un fichier JSON

Pour lire un fichier JSON en Python :

```python
import json

with open('data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

print(data)
```

Gestion des exceptions :

```python
try:
    with open('data.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
except json.JSONDecodeError as e:
    print("Erreur lors du décodage JSON :", e)
except FileNotFoundError:
    print("Fichier non trouvé.")
```

#### Écrire et enregistrer un fichier JSON

Pour écrire des données Python dans un fichier JSON :

```python
import json

data = {
    'name': 'Historien',
    'projets': ['numérisation', 'analyse']
}

with open('output.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)
```

Notes :

- `indent=4` : pour une indentation de 4 espaces, améliore la lisibilité.
- `ensure_ascii=False` : pour conserver les caractères spéciaux (accents, etc.).

---

### La bibliothèque `os`

#### Intérêt de `os`

La bibliothèque `os` permet d'interagir avec le système d'exploitation :

- Gestion des fichiers et répertoires.
- Exécution de commandes système.
- Accès aux variables d'environnement.

#### Création de filepath

Construction de chemins de fichiers de manière indépendante du système :

```python
import os

# Chemin relatif
chemin = os.path.join('dossier', 'sous_dossier', 'fichier.txt')
print(chemin)

# Chemin absolu
chemin_absolu = os.path.abspath(chemin)
print(chemin_absolu)
```

Fonctions utiles :

- `os.path.exists(chemin)` : Vérifie si le chemin existe.
- `os.path.dirname(chemin)` : Renvoie le répertoire parent.
- `os.path.basename(chemin)` : Renvoie le nom du fichier.

#### Création de dossier

Créer un seul dossier :

```python
import os

dossier = 'nouveau_dossier'

if not os.path.exists(dossier):
    os.mkdir(dossier)
    print(f"Dossier '{dossier}' créé.")
else:
    print(f"Dossier '{dossier}' existe déjà.")
```

Créer des répertoires imbriqués :

```python
dossiers = os.path.join('dossier1', 'dossier2', 'dossier3')

if not os.path.exists(dossiers):
    os.makedirs(dossiers)
    print(f"Dossiers '{dossiers}' créés.")
else:
    print(f"Dossiers '{dossiers}' existent déjà.")
```

Gestion des exceptions :

```python
try:
    os.makedirs(dossiers, exist_ok=True)
except OSError as e:
    if e.errno == os.errno.EEXIST:
        print("Le dossier existe déjà.")
    else:
        raise
```

Plus simplement 

```python
os.makedirs(dossiers, exist_ok=True)
```



---

## Télécharger des données avec la bibliothèque `requests`

### Intérêt de la bibliothèque

La bibliothèque `requests` simplifie les requêtes HTTP en Python :

- Plus conviviale que le module intégré `urllib`.
- Gère les sessions, les cookies, l'authentification, etc.
- Bien documentée et largement utilisée.

Installation :

```bash
pip install requests
```

### Manipulation des codes et gestion des exceptions

Comprendre les codes de statut HTTP :

- **200** : OK (succès).
- **404** : Not Found (non trouvé).
- **429** : Too Many Requests
- **500** : Internal Server Error (erreur serveur).

Exemple :

In [ ]:
import requests

response = requests.get('https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/manifest.json')

print("Code de statut :", response.status_code)

Code de statut : 200


### 2.3 Méthode `get` et gérer les `status_code`

Gestion des erreurs :

```python
if response.status_code == 200:
    data = response.json()
else:
    print(f"Erreur {response.status_code} lors de la requête.")
```

Exceptions de `requests` :

```python
import requests

try:
    response = requests.get('https://api.example.com/data')
    response.raise_for_status()  # Lève une exception pour les codes d'erreur HTTP
    data = response.json()
except requests.exceptions.HTTPError as errh:
    print("Erreur HTTP :", errh)
except requests.exceptions.ConnectionError as errc:
    print("Erreur de connexion :", errc)
except requests.exceptions.Timeout as errt:
    print("Délai d'attente dépassé :", errt)
except requests.exceptions.RequestException as err:
    print("Erreur lors de la requête :", err)
```

### Téléchargement avec `requests'

#### Télécharger un manifeste depuis une URL :

In [ ]:
import json
import requests
import os

def download_manifest(manifest_url, output_folder, ms_name):
    # URL du manifeste IIIF
    manifest_url = 'https://www.e-codices.unifr.ch/metadata/iiif/bbb-0318/manifest.json'

    # Télécharger le manifeste
    response = requests.get(manifest_url)

    # Vérifier que la requête s'est bien passée
    if response.status_code == 200:
        manifest = response.json()
        print("Manifeste chargé avec succès.")
        manifest_path = os.path.join(output_folder, ms_name + '_manifest.json')
        
        # Save the manifest.json file in the destination folder
        with open(manifest_path, 'wb') as f:
            f.write(response.content)
        print(f"Manifeste téléchargé dans : {os.path.join(output_folder, ms_name + '_manifest.json')}")
    else:
        print(f"Erreur lors du chargement du manifeste : {response.status_code}")

#### Téléchargement d'image

Télécharger une image depuis une URL :

In [ ]:
import requests

url = 'https://www.e-codices.unifr.ch/loris/bbb/bbb-0318/bbb-0318_001r.jp2/full/full/0/default/jpg'

try:
    response = requests.get(url)
    response.raise_for_status()
    
    # Vérifier le type de contenu
    if url.endswith('jpg'):
        with open('image.jpg', 'wb') as file:
            file.write(response.content)
        print("Image téléchargée avec succès.")
    else:
        print("URL ne pointe pas vers une image.")
except requests.exceptions.RequestException as e:
    print(f"Erreur lors du téléchargement : {e}")


Image téléchargée avec succès.


## Manipuler des images

### La bibliothèque PIL

#### Présentation de PIL/Pillow

- **PIL** : Python Imaging Library, bibliothèque historique pour la manipulation d'images.
- **Pillow** : Fork de PIL, activement maintenu et compatible avec les versions récentes de Python.

Installation :

```bash
pip install Pillow
```

Importation :

```python
from PIL import Image
```

### Manipulation de base d'une image

#### Ouvrir et afficher une image

In [ ]:
from PIL import Image

image = Image.open('PATHTODOWLOAD/image.jpg')
image.show()

#### Redimensionner une image

In [ ]:
nouvelle_image = image.resize((800, 600))
nouvelle_image.save('image_redimensionnee.jpg')

#### Rotation

In [ ]:
image_rotated = image.rotate(90)
image_rotated.save('image_rotated.jpg')

#### Conversion de format

In [ ]:
image.convert('L').save('image_grayscale.jpg')

#### Recadrage

In [31]:
box = (200, 900, 2400, 3500)  # (left, upper, right, lower)
image_cropped = image.crop(box)
image_cropped.save('image_cropped.jpg')

### Récupérer les données des images

#### Accéder aux métadonnées

Informations de base :

In [32]:
print(f"Format : {image.format}")
print(f"Taille : {image.size}")
print(f"Absoluter path : {image.filename}")

Format : JPEG
Taille : (4872, 6496)
Absoluter path : /Users/marioncharpier/Documents/Cours_ENC/Cours_6_Utilisation de l’API IIIF et manipulation de données 2/image.jpg


---

### Conclusion

En maîtrisant ces bibliothèques et techniques, vous serez en mesure de :

- Manipuler des données structurées (JSON).
- Interagir avec le système de fichiers.
- Télécharger des ressources en ligne de manière efficace et sécurisée.
- Traiter et analyser des images pour vos projets en histoire numérique.

Ces compétences sont essentielles pour analyser, conserver et diffuser le patrimoine historique dans le contexte numérique actuel.

---

Vous pouvez utiliser ce notebook comme base pour vos propres projets, en adaptant et en étendant les exemples fournis.

---